# Module 9: Convolutional Neural Networks
## **Lab 3**
### Module coordinator: Aswin Jose and Anirvinya Gururajan

## 1. Understanding Convolutions

<img src="https://miro.medium.com/max/464/0*e-SMFTzO8r7skkpc" width=650px/>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Importing some pytorch packages
import torch
from torch.nn import Conv2d

Central to CNNs, a convolution operation is a linear element-wise multiplication operation between a small filter/kernel and same-sized patch from the image. We move this filter over the image like a sliding window from top left to bottom right. For each point on the image, a value is calculated based on the filter using a convolution operation. These filters can do simplest task like checking if there is a vertical line in the image or complicated task like detecting a human eye in the image.

Let's look at the convolution formula:

Convolution between image
$f(x, y)$ and kernel $k(x, y)$ is
$$f(x,y) * k(x,y) = \sum \limits _{i=0} ^{W-1} \sum \limits _{j=0} ^{H-1} f(i, j) k(x − i, y − j)$$

where $W$ and $H$ are the the width and height of the image.

The code demonstrates the convolution operation of a 2D matrix (image) with various filters

<img src="https://www.researchgate.net/profile/Chaim-Baskin/publication/318849314/figure/fig1/AS:614287726870532@1523469015098/Image-convolution-with-an-input-image-of-size-7-7-and-a-filter-kernel-of-size-3-3.png" alt="Convolution" width=650px height=280px/>

In [ ]:
# 2D 3x3 binary image with vertical edge
image1 = np.array([[1,1,0], [1,1,0], [1,1,0]])

# 2D 3x3 binary image with horizontal edge
image2 = np.array([[0,0,0], [0,0,0], [1,1,1]])

# On plotting the images
plt.imshow(image1, cmap='gray', extent=[0, 3, 3, 0])
plt.show()
plt.imshow(image2, cmap='gray', extent=[0, 3, 3, 0])
plt.show()

In [ ]:
# Vertical Line filter
filter = np.array([[1,0,-1],
                   [1,0,-1],
                   [1,0,-1]])

# Applying filter to first image
output = np.sum(np.multiply(image1, filter))
print('Output from first image: ', output)

# Applying filter to second image
output = np.sum(np.multiply(image2, filter))
print('Output from second image: ', output)

In [ ]:
# Horizontal edge filter
filter = np.array([[-1,-1,-1],
                   [ 0, 0, 0],
                   [ 1, 1, 1]])

output = np.sum(np.multiply(image1, filter))
print('Output from first image: ', output)

output = np.sum(np.multiply(image2, filter))
print('Output from second image: ', output)

Non-zero output suggests that there is a vertical edge present in the first image and not present in the second image. Similarly, horizontal edge is detected in second.

Let's define a function to use convolution layer from Pytorch and use our own kernel to detect edges in image

In [ ]:
def apply_conv(image, kernel, padding=0, stride=1):

  #--------IMAGE PREPROCESSING-------
  image = torch.from_numpy(image)
  # Pytorch requires input to convolution in (N,C,H,W), where N = batch size and C=#channels in input
  input = image.view((1,1,image.shape[0], image.shape[1]))

  # --------------KERNEL-------------
  kernel = torch.from_numpy(kernel.astype(np.float32))

  # Pytorch requires kernel of shape (N,C,H,W), where N = batch size and C=#channels in input
  kernel = kernel.view((1,1,kernel.shape[0], kernel.shape[1]))

  # ---------CONVOLUTION LAYER from Pytorch--------
  conv = Conv2d(in_channels=1, out_channels=1, kernel_size=kernel.shape, padding=padding, stride=stride)

  # Set the kernel weights in the convolution layer
  conv.weight = torch.nn.Parameter(kernel)

  # ---------APPLY CONVOLUTION--------
  output = conv(input.float())
  output_img = output.data.numpy()  # Tensor to back in numpy
  output_img = output_img.reshape((-1, output_img.shape[-1])) # Reshape to 2D image

  return output_img

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:

# Our original lotus image
image = cv2.imread('grid1.jpg', 0)

# Check if image loading was successful
if image is None:
    print("Error: Could not load image. Please check the file path.")
else:
    filter = np.array([[-1, -1, -1],
                       [0, 0, 0],
                       [1, 1, 1]])

    out1 = apply_conv(image, filter, padding=0, stride=1)

    filter = np.array([[1, 0, -1],
                       [1, 0, -1],
                       [1, 0, -1]])

    out2 = apply_conv(image, filter, padding=0, stride=1)

In [ ]:

import cv2
import numpy as np
import matplotlib.pyplot as plt

# Our original lotus image
image = cv2.imread('grid1.jpg', 0)

# Check if image loading was successful
if image is None:
    print("Error: Could not load image. Please check the file path.")
else:
    # Convert image to uint8 if it's not already
    if image.dtype != np.uint8:
        image = image.astype(np.uint8)

    filter = np.array([[-1, -1, -1],
                       [0, 0, 0],
                       [1, 1, 1]])

    out1 = apply_conv(image, filter, padding=0, stride=1)

    filter = np.array([[1, 0, -1],
                       [1, 0, -1],
                       [1, 0, -1]])

    out2 = apply_conv(image, filter, padding=0, stride=1)

## Pooling Layers

Pooling layers are used to reduce the dimensions of the feature maps. Thus, it reduces the number of parameters to learn and the amount of computation performed in the network. The pooling layer summarises the features present in a region of the feature map generated by a convolution layer.

1) Max Pooling:

<img src='https://media.geeksforgeeks.org/wp-content/uploads/20190721025744/Screenshot-2019-07-21-at-2.57.13-AM.png' height=150px/>

2) Average Pooling:

<img src='https://media.geeksforgeeks.org/wp-content/uploads/20190721030705/Screenshot-2019-07-21-at-3.05.56-AM.png' height=150px/>

## Softmax layer/activation
Recall that logistic regression produces a decimal between 0 and 1.0. For example, a logistic regression output of 0.8 from an email classifier suggests an 80% chance of an email being spam and a 20% chance of it being not spam. Clearly, the sum of the probabilities of an email being either spam or not spam is 1.0.

Softmax extends this idea into a multi-class world. That is, Softmax assigns decimal probabilities to each class in a multi-class problem. Those decimal probabilities must add up to 1.0. This additional constraint helps training converge more quickly than it otherwise would.
Softmax is implemented through a neural network layer just before the output layer. The Softmax layer must have the same number of nodes as the output layer.

<img src='https://miro.medium.com/max/1400/1*ReYpdIZ3ZSAPb2W8cJpkBg.jpeg' height=170px />

## Learning to train a CNN network

In [ ]:
# Import packages
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#  Images returned from torchvision dataset classes is in range [0,1]
# We transform them to tensors and normalize them to range [-1,1] using 'Normalize' transform

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

# Classes in CIFAR10
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
print('Training data shape : ', trainset.data.shape, len(trainset.targets))
print('Testing data shape : ', testset.data.shape, len(testset.targets))

# Find the unique numbers from the train labels
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

In [ ]:
def train(num_epochs, model, train_loader, loss_func, optimizer):

  # Training mode
  model.train()

  train_losses = []
  train_acc = []

  # Train the model
  for epoch in range(num_epochs):
    running_loss = 0
    running_acc = 0
    for i, (images, labels) in enumerate(train_loader):

      # clear gradients for this training step
      optimizer.zero_grad()

      # Forward pass
      output = model(images)

      # Calculate loss
      loss = loss_func(output, labels)

      # Backpropagation, compute gradients
      loss.backward()

      # Apply gradients
      optimizer.step()

      # Running loss
      running_loss += loss.item()

      # indices of max probabilities
      _, preds = torch.max(output, dim=1)

      # Calculate number of correct predictions
      correct = (preds.float() == labels).sum()
      running_acc += correct

      # Average loss and acc values
      epoch_loss = running_loss / len(train_loader.dataset)
      epoch_acc = running_acc / len(train_loader.dataset)

    train_losses.append(epoch_loss)
    train_acc.append(epoch_acc)
    print ('Epoch {}/{}, Loss: {:.4f}, Accuracy: {:.4f}'.format(epoch + 1, num_epochs, epoch_loss, epoch_acc*100))

  return train_losses, train_acc

In [ ]:
def test_model(model, testloader):
  correct = 0
  total = 0
  # Deactivate autograd engine (don't compute grads since we're not training)
  with torch.no_grad():
    for data in testloader:
        images, labels = data
        # Calculate outputs by running images through the network
        outputs = model(images)
        # The class with the highest value is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

  print('Accuracy of the network: %d %%' % (
      100 * correct / total))

In [ ]:
# CNN with 2 CONV layers and 3 FC layers
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.fc1 = nn.Linear(32 * 5 * 5, 512)
        self.fc2 = nn.Linear(512, 128)
        # output layer 10 classes
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # flatten all dimensions except batch
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
model = Net()
print(model)

In [ ]:
# Cross Entropy loss for multi-class classification
criterion = nn.CrossEntropyLoss()

In [ ]:
#(), lr:=0.05, momentum=0.9
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)

In [ ]:
# Import necessary libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

# ... (Your existing code for Net class, train function, and test_model function) ...

# Create an instance of the model
model = Net()  # Define the model before using it

# Cross Entropy loss for multi-class classification
criterion = nn.CrossEntropyLoss()

# SGD optimizer with momentum
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)

# ... (Rest of your training and testing code) ...

In [ ]:
# Import necessary libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

# ... (Your existing code for Net class, train function, and test_model function) ...

# Create an instance of the model
model = Net()  # Define the model before using it

# Cross Entropy loss for multi-class classification
criterion = nn.CrossEntropyLoss()

# SGD optimizer with momentum
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)

# ... (Rest of your training and testing code) ...

In [ ]:

# ... previous code ...

# Create an instance of the model
model = Net()

# Cross Entropy loss for multi-class classification
criterion = nn.CrossEntropyLoss()

# SGD optimizer with momentum
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)

# Number of training epochs
num_epochs = 5  # iterations

# Train the model and get the training losses and accuracy
train_losses, train_acc = train(num_epochs, model, trainloader, criterion, optimizer) # Call the train function

# Plotting code
fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(1, 2, 1)
ax.plot(np.arange(1, len(train_losses) + 1), train_losses)
plt.xlabel('Training loss')
plt.ylabel('Epochs')
ax.set_title('Loss vs Epochs')
ax = fig.add_subplot(1, 2, 2)
ax.plot(np.arange(1, len(train_acc) + 1), train_acc)
plt.xlabel('Training accuracy')
plt.ylabel('Epochs')
ax.set_title('Accuracy vs Epochs')
plt.show()

In [ ]:

# Accuracy on test data after training
test_model(model, testloader)
# Import packages
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

# ... (rest of your code) ...

# CNN with 2 CONV layers and 3 FC layers
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.fc1 = nn.Linear(32 * 5 * 5, 512)
        self.fc2 = nn.Linear(512, 128)
        # output layer 10 classes
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # flatten all dimensions except batch
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create an instance of the model
model = Net()

# Cross Entropy loss for multi-class classification
criterion = nn.CrossEntropyLoss()

# SGD optimizer with momentum
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)

# Number of training epochs
num_epochs = 5  # iterations

# Training loop
train_losses, train_acc = train(num_epochs, model, trainloader, criterion, optimizer)


def test_model(model, testloader):
  correct = 0
  total = 0
  # Deactivate autograd engine (don't compute grads since we're not training)
  with torch.no_grad():
    for data in testloader:
        images, labels = data
        # Calculate outputs by running images through the network
        outputs = model(images)
        # The class with the highest value is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

  print('Accuracy of the network: %d %%' % (
      100 * correct / total))

# ... (rest of your code) ...

# Accuracy on test data after training
test_model(model, testloader)

## Questions
1) List some reasons why we should prefer CNN over ANN for image classification?

2) Try improving the CNN performance further by tuning the hyperparameters(epochs, optimizer, LR etc). Report the improved test accuracy.

3) What happens if you reduce the number of convolution layers to only 1?

4) Why didn't we use the Softmax activation in the last layer of CNN?

5) How does pooling help in this context ?

1.List some reasons why we should prefer CNN over ANN for image classification?

Answer:

CNN vs ANN for Image Classification
Here are some reasons why CNNs are preferred over ANNs for image classification:

1. *Spatial Hierarchy*: CNNs can capture spatial hierarchies of features in images, which is essential for image classification.
2. *Local Connectivity*: CNNs use local connectivity, which allows them to focus on specific regions of the image.
3. *Shared Weights*: CNNs use shared weights, which reduces the number of parameters and improves generalization.
4. *Translation Equivariance*: CNNs are translation equivariant, meaning that they can recognize features regardless of their position in the image.
5. *Robustness to Variations*: CNNs can be robust to variations in image scale, rotation, and lighting condition

Conclusion
CNNs are well-suited for image classification tasks due to their ability to capture spatial hierarchies, local connectivity, and shared weights. These properties make CNNs more effective and efficient than ANNs for image classification tasks.

2.Try improving the CNN performance further by tuning the hyperparameters(epochs, optimizer, LR etc).


Answer:
Hyperparameter Tuning
To improve the CNN performance, we can try tuning the hyperparameters:

Hyperparameters to Tune
1. *Epochs*: Increase or decrease the number of epochs to find the optimal value.
2. *Optimizer*: Try different optimizers such as Adam, RMSprop, or SGD.
3. *Learning Rate (LR)*: Adjust the learning rate to find the optimal value.
4. *Batch Size*: Experiment with different batch sizes.

Code
```
from tensorflow.keras.optimizers import Adam, RMSprop, SGD

Define hyperparameter space
epochs = [10, 20, 30]
optimizers = [Adam(), RMSprop(), SGD()]
learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [32, 64, 128]

Perform grid search
best_accuracy = 0
best_hyperparams = None

for epoch in epochs:
    for optimizer in optimizers:
        for lr in learning_rates:
            for batch_size in batch_sizes:
                # Set hyperparameters
                model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
                optimizer.lr.assign(lr)
                
                # Train model
                history = model.fit(X_train, y_train, epochs=epoch, batch_size=batch_size, validation_data=(X_test, y_test))
                
                # Evaluate model
                test_loss, test_acc = model.evaluate(X_test, y_test)
                
                # Update best hyperparameters
                if test_acc > best_accuracy:
                    best_accuracy = test_acc
                    best_hyperparams = {'epochs': epoch, 'optimizer': optimizer, 'lr': lr, 'batch_size': batch_size}

print(f"Best Hyperparameters: {best_hyperparams}")
print(f"Best Accuracy: {best_accuracy}")
```

Random Search
Alternatively, you can use random search to find the optimal hyperparameters:

```
import random

Define hyperparameter space
epochs = [10, 20, 30]
optimizers = [Adam(), RMSprop(), SGD()]
learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [32, 64, 128]

Perform random search
best_accuracy = 0
best_hyperparams = None

for _ in range(10):
    # Randomly select hyperparameters
    epoch = random.choice(epochs)
    optimizer = random.choice(optimizers)
    lr = random.choice(learning_rates)
    batch_size = random.choice(batch_sizes)
    
    # Set hyperparameters
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    optimizer.lr.assign(lr)
    
    # Train model
    history = model.fit(X_train, y_train, epochs=epoch, batch_size=batch_size, validation_data=(X_test, y_test))
    
    # Evaluate model
    test_loss, test_acc = model.evaluate(X_test, y_test)
    
    # Update best hyperparameters
    if test_acc > best_accuracy:
        best_accuracy = test_acc
        best_hyperparams = {'epochs': epoch, 'optimizer': optimizer, 'lr': lr, 'batch_size': batch_size}

print(f"Best Hyperparameters: {best_hyperparams}")
print(f"Best Accuracy: {best_accuracy}")
```

Conclusion
Hyperparameter tuning can significantly improve the performance of the CNN model. By experimenting with different hyperparameters, you can find the optimal combination that results in the best accuracy.

3.What happens if you reduce the number of convolution layers to only 1?

Answer:

Reducing Convolution Layers
If you reduce the number of convolution layers to only 1:

Effects
1. *Reduced Feature Extraction*: With only one convolution layer, the model may not be able to extract complex features from the input data.
2. *Decreased Representation Power*: A single convolution layer may not be sufficient to capture the underlying patterns and relationships in the data.
3. *Potential Underfitting*: The model may underfit the training data, resulting in poor performance on both the training and testing sets.

Possible Outcomes
1. *Lower Accuracy*: The model's accuracy may decrease due to the reduced ability to extract features and represent complex relationships.
2. *Increased Bias*: The model may exhibit increased bias towards certain features or patterns, leading to poor generalization.

When to Use a Single Convolution Layer
1. *Simple Problems*: A single convolution layer may be sufficient for simple image classification problems with limited complexity.
2. *Limited Data*: When working with limited data, a single convolution layer may be a good starting point to avoid overfitting.

Conclusion
Reducing the number of convolution layers to only 1 can negatively impact the model's performance, especially for complex image classification tasks. However, it may be suitable for simple problems or when working with limited data.

4.Why didn't we use the Softmax activation in the last layer of CNN?



Answer:

Softmax Activation
In the CNN model, we used the Softmax activation function implicitly through the `sparse_categorical_crossentropy` loss function.

Why Not Explicit Softmax
When using `sparse_categorical_crossentropy` or `categorical_crossentropy` loss functions in Keras, you don't need to explicitly define a Softmax activation function in the last layer.

How it Works
The Softmax activation function is applied internally by the loss function, which expects logits (unnormalized scores) as input.

Explicit Softmax
If you want to explicitly define a Softmax activation function, you can use it in the last layer:

```
model.add(Dense(num_classes, activation='softmax'))
```

In this case, you would use `categorical_crossentropy` loss function.

Conclusion
The choice between explicit and implicit Softmax activation depends on the loss function used. Both approaches can work, but it's essential to understand how the loss function interacts with the activation function.

5.Robustness to Variations: CNNs can be robust to variations in image scale, rotation, and lighting condition

Answer:

Robustness to Variations
CNNs can be robust to variations in:

1. *Image Scale*: CNNs can learn to recognize objects at different scales due to the use of convolutional and pooling layers.
2. *Rotation*: While traditional CNNs are not inherently rotation-invariant, techniques like data augmentation and rotation-invariant layers can improve robustness to rotation.
3. *Lighting Conditions*: CNNs can learn to recognize objects under different lighting conditions due to the use of convolutional layers that extract features from local regions.

Achieving Robustness
To achieve robustness to variations, CNNs can utilize:

1. *Data Augmentation*: Applying random transformations (e.g., rotation, scaling, flipping) to training images.
2. *Regularization Techniques*: Using techniques like dropout and weight decay to prevent overfitting.
3. *Robust Loss Functions*: Using loss functions that are robust to outliers and variations.

Limitations
While CNNs can be robust to some variations, they may still struggle with:

1. *Severe Occlusions*: Objects that are heavily occluded may be difficult for CNNs to recognize.
2. *Large Viewpoint Changes*: Objects viewed from drastically different angles may require specialized architectures or training.

Conclusion
CNNs can be robust to various image variations, but achieving robustness often requires careful design, training, and regularization techniques.